In [1]:
import pandas as pd
import math
from math import log2

In [2]:
tennis_pd = pd.read_csv('tennis.csv')

In [3]:
tennis_pd

,Day,Outlook,Temp.,Humidity,Wind,Decision
0,1,Sunny,Hot,High,Weak,No
1,2,Sunny,Hot,High,Strong,No
2,3,Overcast,Hot,High,Weak,Yes
3,4,Rain,Mild,High,Weak,Yes
4,5,Rain,Cool,Normal,Weak,Yes
5,6,Rain,Cool,Normal,Strong,No
6,7,Overcast,Cool,Normal,Strong,Yes
7,8,Sunny,Mild,High,Weak,No
8,9,Sunny,Cool,Normal,Weak,Yes
9,10,Rain,Mild,Normal,Weak,Yes


# Computing Entropy
(1) Write a function that computes the Entropy of a set 𝑆 with 𝑁𝑝𝑜𝑠 positive observations and 𝑁𝑛𝑒𝑔 negative
observations.


In [4]:
# Compute probability
def prob_compute (class_freq,tot_count):
    return class_freq/tot_count

# Compute Entropy from raw frequency counts (Valid for Binary attribute yes-no)
def compute_raw_entropy(val1,val2):
    # Handle the zero-probability
    if (val1==0 or val2==0):
        return 0
    else:
        p1 = prob_compute(val1,val1+val2)
        p2 = prob_compute(val2,val1+val2)    
        return -1 * (p1*log2(p1) + p2*log2(p2))

# Compute Entropy from a set S with val1=Nneg, val2=Npos    
def compute_entropy_from_set(subset_pd,target_column):
    group_freq = subset_pd.groupby([target]).size()
    if (len(group_freq)<2):
        return 0
    else:
        return compute_raw_entropy(group_freq[0],group_freq[1])    

**This computes the entropy for the Decision {9-Yes, 5-No}**

In [85]:
target='Decision'    
subset_pd = tennis_pd
# display(group_freq)
print('Entropy for Decision to play is:', compute_entropy_from_set\
      (subset_pd=tennis_pd,target_column='Decision'))


Entropy for Decision to play is: 0.9402859586706309


## Task 2: Computing Information Gain
(2) Write a function that takes as input a set 𝑆 of observations and an attribute 𝐴 from these observations, and
calculates the Information Gain, denoted as Gain(S,A), as if we were to split on that attribute in the context of the
ID3 decision tree algorithm.

In [84]:
attribute_name = 'Outlook'
subset_pd = tennis_pd
target = 'Decision'
def compute_ig(subset_pd,attribute_name,target):
    class_split = subset_pd.groupby(attribute_name)
    subset_freq = subset_pd.groupby([target]).size()
    subset_entropy = compute_raw_entropy(subset_freq[0],subset_freq[1])
    print('E(%s): ' % (target) , subset_entropy)
    group_sum = 0
    for name, group in class_split:
        class_total = group.shape[0]
        class_weight = class_total/subset_pd.shape[0] 
        group_sum += class_weight * \
        compute_entropy_from_set(subset_pd=group,target_column=target)
        print('E(%s|%s=%s):' % (target,attribute_name,name),\
              compute_entropy_from_set(subset_pd=group,target_column=target))
    print('Information Gain for %s, Gain(S=%s|A=%s): %s' % \
          (attribute_name,target,attribute_name, subset_entropy - group_sum))
    return subset_entropy - group_sum

attribute_name = 'Outlook'
subset_pd = tennis_pd
target = 'Decision'
def compute_ig_silent(subset_pd,attribute_name,target):
    class_split = subset_pd.groupby(attribute_name)
    subset_freq = subset_pd.groupby([target]).size()
    subset_entropy = compute_raw_entropy(subset_freq[0],subset_freq[1])
    group_sum = 0
    for name, group in class_split:
        class_total = group.shape[0]
        class_weight = class_total/subset_pd.shape[0] 
        group_sum += class_weight * \
        compute_entropy_from_set(subset_pd=group,target_column=target)
    return subset_entropy - group_sum



## Estimate Information Gain of all Attributes
(3) Estimate the Information Gain of all the attributes. Which one would you choose for the root node of your
decision tree? 

In [7]:
compute_ig(subset_pd=tennis_pd,attribute_name='Outlook',target='Decision')

E(Decision):  0.9402859586706309
E(Decision|Outlook=Overcast): 0
E(Decision|Outlook=Rain): 0.9709505944546686
E(Decision|Outlook=Sunny): 0.9709505944546686
Information Gain for Outlook, Gain(S=Decision|A=Outlook): 0.2467498197744391


0.2467498197744391

In [8]:
compute_ig(subset_pd=tennis_pd,attribute_name='Temp.',target='Decision')

E(Decision):  0.9402859586706309
E(Decision|Temp.=Cool): 0.8112781244591328
E(Decision|Temp.=Hot): 1.0
E(Decision|Temp.=Mild): 0.9182958340544896
Information Gain for Temp., Gain(S=Decision|A=Temp.): 0.029222565658954647


0.029222565658954647

In [9]:
compute_ig(subset_pd=tennis_pd,attribute_name='Humidity',target='Decision')

E(Decision):  0.9402859586706309
E(Decision|Humidity=High): 0.9852281360342515
E(Decision|Humidity=Normal): 0.5916727785823275
Information Gain for Humidity, Gain(S=Decision|A=Humidity): 0.15183550136234136


0.15183550136234136

In [10]:
compute_ig(subset_pd=tennis_pd,attribute_name='Wind',target='Decision')

E(Decision):  0.9402859586706309
E(Decision|Wind=Strong): 1.0
E(Decision|Wind=Weak): 0.8112781244591328
Information Gain for Wind, Gain(S=Decision|A=Wind): 0.04812703040826927


0.04812703040826927



### As shown above, **OUTLOOK** had the highest information gain and should be chosen as root node

# Task 3 - Decision Tree Algorithm

Implement the ID3 Decision Tree from the pseudocode (recursive algorithm) below and induce/learn the tree from
the data in the Figure. 

### Calculate the Information Gain

### Get the attribute that has the highest Information Gain
Let's make a single pass of the recursive process

In [132]:
temp_list = []
remaining_columns = tennis_pd.columns[1:5]
for col_name in remaining_columns:
    print(col_name)
    temp_ig = compute_ig_silent(\
    subset_pd=tennis_pd,attribute_name=col_name,target='Decision')
    temp_list.append(temp_ig)
    
top_ig = remaining_columns[temp_list.index(max(temp_list))]    
root = Tree(top_ig)
remaining_columns = remaining_columns.drop(top_ig)
print('Highest IG is:', top_ig)
print('Remaining Columns for next recursion are:',remaining_columns.values)

Outlook
Temp.
Humidity
Wind
Highest IG is: Outlook
Remaining Columns for next recursion are: ['Temp.' 'Humidity' 'Wind']


In [129]:
class Tree(object):
    def __init__(self,attribute):
        self.left = None
        self.right = None
        self.attribute = attribute
        self.label = None
        self.r_value = None
        self.l_value = None
        self.child=[]
        
    def createChildren(self,attribute_count):
        for i in range(0,attribute_count):
            self.child.append(Tree())
            
    def addChild(self,attribute):
        self.child.append(Tree(attribute))            
 
    def setLabel(self,label):
        self.label=label

### Compute IG for remaining attributes
Now for the recursive part... integrate this to the previous step

In [135]:
initial_columns = tennis_pd.columns[1:-1]
root_node = Tree(attribute='root')
def id3_recursion(input_pd,initial_columns):
    remaining_columns = initial_columns
    temp_list = []    
    if len(remaining_columns)>0:
        if(len(input_pd.groupby(['Decision'])['Decision'])==1):
            print('Target Decision is purely: ', \
                  input_pd.groupby(['Decision'])['Decision'].\
                  unique()[0],'Return LEAF NODE')
            leaf_node = Tree(attribute=input_pd.groupby(\
                    ['Decision'])['Decision'].unique()[0])
            leaf_node.label=input_pd.groupby(\
                    ['Decision'])['Decision'].unique()[0]
            return leaf_node
        for col_name in remaining_columns:
            print(col_name)
            temp_ig = compute_ig_silent(subset_pd=tennis_pd,\
                                        attribute_name=col_name,target='Decision')
            temp_list.append(temp_ig)
            print('Information Gain for %s = %.3f' % (col_name,temp_ig))
        top_ig = initial_columns[temp_list.index(max(temp_list))]  
        print('Highest IG is :',top_ig)
        initial_columns = initial_columns.drop(top_ig)
        print('Choose next attribute from: ',initial_columns)
        curr_col = top_ig
        attr_val = input_pd.groupby(curr_col)[curr_col]
        print('Attribute values for',attr_val.unique())
        #         curr_split = attr_val.unique()[0][0]       
        root_node.addChild(Tree(attribute=top_ig))
        for curr_split in attr_val.unique():
            print('################# Split Data for: %s= %s #################'\
                  % ( top_ig,curr_split[0]))
            id3_recursion(input_pd[input_pd[curr_col]==\
                                   curr_split[0]],initial_columns=initial_columns)

In [136]:
id3_recursion(tennis_pd,initial_columns=initial_columns)

Outlook
Information Gain for Outlook = 0.247
Temp.
Information Gain for Temp. = 0.029
Humidity
Information Gain for Humidity = 0.152
Wind
Information Gain for Wind = 0.048
Highest IG is : Outlook
Choose next attribute from:  Index(['Temp.', 'Humidity', 'Wind'], dtype='object')
Attribute values for Outlook
Overcast    [Overcast]
Rain            [Rain]
Sunny          [Sunny]
Name: Outlook, dtype: object
################# Split Data for: Outlook= Overcast #################
Target Decision is purely:  ['Yes'] Return LEAF NODE
################# Split Data for: Outlook= Rain #################
Temp.
Information Gain for Temp. = 0.029
Humidity
Information Gain for Humidity = 0.152
Wind
Information Gain for Wind = 0.048
Highest IG is : Humidity
Choose next attribute from:  Index(['Temp.', 'Wind'], dtype='object')
Attribute values for Humidity
High        [High]
Normal    [Normal]
Name: Humidity, dtype: object
################# Split Data for: Humidity= High #################
Temp.
Information G

In [151]:
for i in root_node.child:
    print(i.attribute)
    print(root_node.attribute , '-->', i.attribute)

root --> <__main__.Tree object at 0x7f2a4a0a0358>
root --> <__main__.Tree object at 0x7f2a4a05bbe0>
root --> <__main__.Tree object at 0x7f2a4a05be48>
root --> <__main__.Tree object at 0x7f2a4a0f03c8>
root --> <__main__.Tree object at 0x7f2a4a0f0278>
